# CatBoost - 피처 54

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "./input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/lg-aimers/sample_submission.csv
/kaggle/input/lg-aimers/train.csv
/kaggle/input/lg-aimers/test.csv


In [2]:
data_path = '/kaggle/input/lg-aimers/'

train = pd.read_csv(data_path + 'train.csv', index_col= 'ID')
test = pd.read_csv(data_path + 'test.csv', index_col= 'ID')
submission = pd.read_csv(data_path + 'sample_submission.csv', index_col= 'ID')

## 1. 데이터 통합

In [3]:
all_data = pd.concat([train, test], ignore_index= True)
all_data = all_data.drop('임신 성공 여부', axis= 1)

In [4]:
drop_features = []

In [5]:
# "특정 시술 유형" 제거
# "주요 시술 유형" 생성

cat_features = [ 
    "시술 당시 나이", # FI top4
    "시술 유형",
    "주요 시술 유형",
    "배란 유도 유형",
    "난자 출처", # FI top
        
    "난자 기증자 나이", #FI top (알수 없음)

    "정자 출처",
    "정자 기증자 나이",
    "시술 시기 코드", 
    "배아 생성 주요 이유"
]

print(len(cat_features))

10


In [6]:
# "IVF 시술 횟수", "IVF 임신 횟수", "IVF 출산 횟수" 제거
# "DI 시술 횟수", "DI 임신 횟수", "DI 출산 횟수" 제거
# "IVF 임신 성공률", "IVF 출산 성공률", "IVF 실패 횟수", "IVF 유산 횟수", "DI 임신 성공률", "DI 출산 성공률", "DI 실패 횟수", "DI 유산 횟수" 생성 

ord_features = [
    "IVF 임신 성공률", 
    "IVF 출산 성공률", 
    "IVF 실패 횟수", 
    "IVF 유산 횟수", 
    "DI 임신 성공률", 
    "DI 출산 성공률", 
    "DI 실패 횟수", 
    "DI 유산 횟수",
    
    "클리닉 내 총 시술 횟수", #FI top
    "총 시술 횟수",
    "총 임신 횟수",
    "총 출산 횟수"
]

print(len(ord_features))

# drop_features = drop_features + ["총 시술 횟수", "총 임신 횟수", "총 출산 횟수"]

12


In [7]:
num_features = [
    "이식된 배아 수", # FI top1
    "해동 난자 수", #FI top
    "수집된 신선 난자 수", #FI top7
    "파트너 정자와 혼합된 난자 수", #FI top
    "배아 이식 경과일", # FI top2,
    
    
    "저장된 배아 수", # FI top3
    "미세주입 후 저장된 배아 수", #FI top
    "총 생성 배아 수", # FI top5
    "미세주입된 난자 수", #FI top8
    "혼합된 난자 수", #FI top
    "미세주입에서 생성된 배아 수", #FI top
    "미세주입 배아 이식 수", #FI top
    "해동된 배아 수", #FI top
    "난자 채취 경과일", #FI top
    "임신 시도 또는 마지막 임신 경과 연수", #FI top

    "난자 해동 경과일",
    "난자 혼합 경과일",
    "배아 해동 경과일",
    "저장된 신선 난자 수",
    "기증자 정자와 혼합된 난자 수"
]

print(len(num_features))

# drop_features = drop_features + ["난자 해동 경과일", "난자 혼합 경과일", "배아 해동 경과일", "저장된 신선 난자 수", "기증자 정자와 혼합된 난자 수"]

20


In [8]:
# "남성 주 불임 원인", "남성 부 불임 원인", "불임 원인 - 남성 요인", "불임 원인 - 정자 농도", "불임 원인 - 정자 면역학적 요인", "불임 원인 - 정자 운동성", "불임 원인 - 정자 형태",
# "여성 주 불임 원인", "여성 부 불임 원인", "불임 원인 - 난관 질환", "불임 원인 - 배란 장애", "불임 원인 - 여성 요인", "불임 원인 - 자궁경부 문제", "불임 원인 - 자궁내막증" 제거
# "부부 주 불임 원인", "부부 부 불임 원인" 제거
# "남성 불임 원인", "여성 불임 원인", "부부 불임 원인" 생성
# "PGD 시술 여부", "PGS 시술 여부" 제거 

bin_features = [
    '배란 자극 여부',
    '단일 배아 이식 여부', #FI top
    '착상 전 유전 진단 사용 여부',
    '남성 불임 원인',
    '여성 불임 원인',
    '부부 불임 원인',
    "동결 배아 사용 여부",
    "신선 배아 사용 여부",

    "불명확 불임 원인",
    "기증 배아 사용 여부",
    '착상 전 유전 검사 사용 여부',
    "대리모 여부"
]

print(len(bin_features))

drop_features = drop_features + ["PGD 시술 여부", "PGS 시술 여부"]

12


In [9]:
# 시술 유형(cat feature) 피처 엔지니어링 

all_data["주요 시술 유형"] = all_data["특정 시술 유형"].apply(lambda x: 
    "ICSI" if "ICSI" in str(x) else 
    "IVF" if "IVF" in str(x) else 
    "IUI" if "IUI" in str(x) else 
    "Other"
)

drop_features = drop_features + ['특정 시술 유형']

all_data['주요 시술 유형'].value_counts(normalize=True, dropna=False) * 100

주요 시술 유형
ICSI     50.208130
IVF      36.909168
Other    10.511578
IUI       2.371124
Name: proportion, dtype: float64

In [10]:
# 과거 이력(ord feature) 피처 엔지니어링

past_features = ["IVF 시술 횟수", "DI 시술 횟수", "IVF 임신 횟수", "DI 임신 횟수", "IVF 출산 횟수", "DI 출산 횟수"]

for col in ["총 시술 횟수", "총 임신 횟수", "총 출산 횟수", "클리닉 내 총 시술 횟수"]:
    all_data[col] = all_data[col].str.replace("회 이상", "").str.replace("회", "").astype(float)
        
for col in past_features:
    all_data[col] = all_data[col].str.replace("회 이상", "").str.replace("회", "").astype(float)

all_data["IVF 임신 성공률"] = all_data["IVF 임신 횟수"] / (all_data["IVF 시술 횟수"] + 1)
all_data["IVF 출산 성공률"] = all_data["IVF 출산 횟수"] / (all_data["IVF 임신 횟수"] + 1)
all_data["IVF 실패 횟수"] = all_data["IVF 시술 횟수"] - all_data["IVF 임신 횟수"]
all_data["IVF 유산 횟수"] = all_data["IVF 임신 횟수"] - all_data["IVF 출산 횟수"]

all_data["DI 임신 성공률"] = all_data["DI 임신 횟수"] / (all_data["DI 시술 횟수"] + 1)
all_data["DI 출산 성공률"] = all_data["DI 출산 횟수"] / (all_data["DI 임신 횟수"] + 1)
all_data["DI 실패 횟수"] = all_data["DI 시술 횟수"] - all_data["DI 임신 횟수"]
all_data["DI 유산 횟수"] = all_data["DI 임신 횟수"] - all_data["DI 출산 횟수"]

drop_features = drop_features + past_features

In [11]:
# 불임 원인(bin features) 피처 엔지니어링 

infertility_features = ["남성 주 불임 원인", "남성 부 불임 원인", "여성 주 불임 원인", "여성 부 불임 원인",
                        "불임 원인 - 남성 요인", "불임 원인 - 정자 농도", "불임 원인 - 정자 면역학적 요인", "불임 원인 - 정자 운동성", "불임 원인 - 정자 형태",
                        "불임 원인 - 난관 질환", "불임 원인 - 배란 장애", "불임 원인 - 여성 요인", "불임 원인 - 자궁경부 문제", "불임 원인 - 자궁내막증",
                        "부부 주 불임 원인", "부부 부 불임 원인"]

all_data["남성 불임 원인"] = (
    all_data["남성 주 불임 원인"] + 
    all_data["남성 부 불임 원인"] + 
    all_data["불임 원인 - 남성 요인"] + 
    all_data["불임 원인 - 정자 농도"] + 
    all_data["불임 원인 - 정자 면역학적 요인"] + 
    all_data["불임 원인 - 정자 운동성"] + 
    all_data["불임 원인 - 정자 형태"]
).apply(lambda x: 1 if x > 0 else 0)

all_data["여성 불임 원인"] = (
    all_data["여성 주 불임 원인"] + 
    all_data["여성 부 불임 원인"] + 
    all_data["불임 원인 - 난관 질환"] + 
    all_data["불임 원인 - 배란 장애"] + 
    all_data["불임 원인 - 여성 요인"] + 
    all_data["불임 원인 - 자궁경부 문제"] + 
    all_data["불임 원인 - 자궁내막증"]
).apply(lambda x: 1 if x > 0 else 0)

all_data["부부 불임 원인"] = (
    all_data["부부 주 불임 원인"] + 
    all_data["부부 부 불임 원인"]
).apply(lambda x: 1 if x > 0 else 0)

drop_features = drop_features + infertility_features

print(all_data['남성 불임 원인'].value_counts(normalize=True, dropna=False) * 100)
print(all_data['여성 불임 원인'].value_counts(normalize=True, dropna=False) * 100)
print(all_data['부부 불임 원인'].value_counts(normalize=True, dropna=False) * 100)

남성 불임 원인
0    60.57855
1    39.42145
Name: proportion, dtype: float64
여성 불임 원인
0    66.416583
1    33.583417
Name: proportion, dtype: float64
부부 불임 원인
0    95.81546
1     4.18454
Name: proportion, dtype: float64


In [12]:
extracted_data = all_data.drop(columns=drop_features, errors='ignore')

In [13]:
extracted_features = extracted_data.columns
extracted_features

Index(['시술 시기 코드', '시술 당시 나이', '임신 시도 또는 마지막 임신 경과 연수', '시술 유형', '배란 자극 여부',
       '배란 유도 유형', '단일 배아 이식 여부', '착상 전 유전 검사 사용 여부', '착상 전 유전 진단 사용 여부',
       '불명확 불임 원인', '배아 생성 주요 이유', '총 시술 횟수', '클리닉 내 총 시술 횟수', '총 임신 횟수',
       '총 출산 횟수', '총 생성 배아 수', '미세주입된 난자 수', '미세주입에서 생성된 배아 수', '이식된 배아 수',
       '미세주입 배아 이식 수', '저장된 배아 수', '미세주입 후 저장된 배아 수', '해동된 배아 수', '해동 난자 수',
       '수집된 신선 난자 수', '저장된 신선 난자 수', '혼합된 난자 수', '파트너 정자와 혼합된 난자 수',
       '기증자 정자와 혼합된 난자 수', '난자 출처', '정자 출처', '난자 기증자 나이', '정자 기증자 나이',
       '동결 배아 사용 여부', '신선 배아 사용 여부', '기증 배아 사용 여부', '대리모 여부', '난자 채취 경과일',
       '난자 해동 경과일', '난자 혼합 경과일', '배아 이식 경과일', '배아 해동 경과일', '주요 시술 유형',
       'IVF 임신 성공률', 'IVF 출산 성공률', 'IVF 실패 횟수', 'IVF 유산 횟수', 'DI 임신 성공률',
       'DI 출산 성공률', 'DI 실패 횟수', 'DI 유산 횟수', '남성 불임 원인', '여성 불임 원인',
       '부부 불임 원인'],
      dtype='object')

In [14]:
len(extracted_features)

54

## 2. 피처 엔지니어링: 원 핫 인코딩

In [15]:
from sklearn.preprocessing import OneHotEncoder

onehot_encoder = OneHotEncoder()

encoded_cat_matrix = onehot_encoder.fit_transform(all_data[cat_features])

encoded_cat_matrix

<346418x57 sparse matrix of type '<class 'numpy.float64'>'
	with 3464180 stored elements in Compressed Sparse Row format>

In [16]:
remaining_features = list(set(extracted_features) - set(cat_features))

In [17]:
print(extracted_data[remaining_features].dtypes)

클리닉 내 총 시술 횟수            float64
DI 임신 성공률                float64
IVF 출산 성공률               float64
착상 전 유전 검사 사용 여부         float64
미세주입에서 생성된 배아 수          float64
DI 실패 횟수                 float64
DI 출산 성공률                float64
여성 불임 원인                   int64
신선 배아 사용 여부              float64
저장된 신선 난자 수              float64
기증 배아 사용 여부              float64
난자 채취 경과일                float64
이식된 배아 수                 float64
배아 이식 경과일                float64
총 시술 횟수                  float64
난자 혼합 경과일                float64
DI 유산 횟수                 float64
부부 불임 원인                   int64
임신 시도 또는 마지막 임신 경과 연수    float64
난자 해동 경과일                float64
IVF 유산 횟수                float64
수집된 신선 난자 수              float64
배아 해동 경과일                float64
총 출산 횟수                  float64
배란 자극 여부                   int64
총 임신 횟수                  float64
혼합된 난자 수                 float64
IVF 실패 횟수                float64
남성 불임 원인                   int64
미세주입 후 저장된 배아 수          float64
착상 전 유전 진단

In [18]:
from scipy import sparse

extracted_data_sprs = sparse.hstack([sparse.csr_matrix(extracted_data[remaining_features].fillna(0)),
                               encoded_cat_matrix],
                              format='csr')

## 3. 데이터 나누기

In [19]:
num_train = len(train) 

X = extracted_data_sprs[:num_train]
X_test = extracted_data_sprs[num_train:]

y = train['임신 성공 여부'].values

In [20]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=0)

In [21]:
from catboost import CatBoostClassifier, Pool

# CatBoost 전용 데이터셋
bayes_dtrain = Pool(X_train, y_train, cat_features=None)
bayes_dvalid = Pool(X_valid, y_valid, cat_features=None)

## 4. 층화 K 폴드

In [22]:
from sklearn.model_selection import StratifiedKFold

# 층화 K 폴드 교차 검증기
folds = StratifiedKFold(n_splits=10, shuffle=True, random_state=1991)

## 5. 베이지안 최적화

In [23]:
# 베이지안 최적화를 위한 하이퍼파라미터 범위
param_bounds = {
    'depth': (4, 10),
    'learning_rate': (0.01, 0.1),
    'l2_leaf_reg': (1, 10),
    'bagging_temperature': (0.1, 1.0),
    'border_count': (32, 255)
}

# 고정된 하이퍼파라미터
fixed_params = {
    'loss_function': 'Logloss',
    'eval_metric': 'AUC',
    'iterations': 2500,
    'random_seed': 1991,
    'verbose': 200
}

In [24]:
from sklearn.metrics import roc_auc_score

# 평가 함수 정의
def eval_function(depth, learning_rate, l2_leaf_reg, bagging_temperature, border_count):
    params = {
        'depth': int(round(depth)),
        'learning_rate': learning_rate,
        'l2_leaf_reg': l2_leaf_reg,
        'bagging_temperature': bagging_temperature,
        'border_count': int(round(border_count))
    }
    
    params.update(fixed_params)
    print('하이퍼파라미터:', params)
    
    # CatBoost 모델 훈련
    cat_model = CatBoostClassifier(**params)
    cat_model.fit(bayes_dtrain, 
                  eval_set=bayes_dvalid, 
                  early_stopping_rounds=100, 
                  verbose=0)
    
    # 검증 데이터 예측
    preds = cat_model.predict_proba(X_valid)[:, 1]
    
    # ROC-AUC 계산
    roc_auc = roc_auc_score(y_valid, preds)
    print(f'ROC-AUC : {roc_auc}\n')
    
    return roc_auc

In [25]:
from bayes_opt import BayesianOptimization

# 베이지안 최적화 객체 생성
optimizer = BayesianOptimization(f=eval_function, pbounds=param_bounds, random_state=0)

In [26]:
# 베이지안 최적화 수행
# TODO: init_points 10~15, n_iter 30~70
optimizer.maximize(init_points=10, n_iter=50)

|   iter    |  target   | baggin... | border... |   depth   | l2_lea... | learni... |
-------------------------------------------------------------------------------------
하이퍼파라미터: {'depth': 8, 'learning_rate': 0.048128931940501433, 'l2_leaf_reg': 5.903948646972072, 'bagging_temperature': 0.5939321535345923, 'border_count': 191, 'loss_function': 'Logloss', 'eval_metric': 'AUC', 'iterations': 2500, 'random_seed': 1991, 'verbose': 200}
ROC-AUC : 0.7409769110525363

| 1         | 0.741     | 0.5939    | 191.5     | 7.617     | 5.904     | 0.04813   |
하이퍼파라미터: {'depth': 9, 'learning_rate': 0.04450973669432, 'l2_leaf_reg': 9.672964844509263, 'bagging_temperature': 0.6813047017599905, 'border_count': 130, 'loss_function': 'Logloss', 'eval_metric': 'AUC', 'iterations': 2500, 'random_seed': 1991, 'verbose': 200}
ROC-AUC : 0.7412414399709103

| 2         | 0.7412    | 0.6813    | 129.6     | 9.351     | 9.673     | 0.04451   |
하이퍼파라미터: {'depth': 7, 'learning_rate': 0.016393245237809825, 'l2_lea

In [27]:
max_params = optimizer.max['params']
max_params['depth'] = int(round(max_params['depth']))
max_params['border_count'] = int(round(max_params['border_count']))
max_params.update(fixed_params)

In [28]:
max_params

{'bagging_temperature': 0.6609529183051456,
 'border_count': 211,
 'depth': 7,
 'l2_leaf_reg': 9.216098157994194,
 'learning_rate': 0.05606213285706964,
 'loss_function': 'Logloss',
 'eval_metric': 'AUC',
 'iterations': 2500,
 'random_seed': 1991,
 'verbose': 200}

In [29]:
# max_params_catboost = {
# 	'bagging_temperature': 0.8125525342743981,
# 	'border_count': 150,
# 	'depth': 7,
# 	'l2_leaf_reg': 9.330369744633948,
# 	'learning_rate': 0.016393245237809825,
# 	'loss_function': 'Logloss',
# 	'eval_metric': 'AUC',
# 	'iterations': 2500,
# 	'random_seed': 1991,
# 	'verbose': 200
# }

In [30]:
from sklearn.metrics import roc_auc_score

# OOF 방식으로 모델 훈련, 검증, 예측
oof_val_preds = np.zeros(X.shape[0])
oof_test_preds = np.zeros(X_test.shape[0])

for idx, (train_idx, valid_idx) in enumerate(folds.split(X, y)):
    print('#'*40, f'폴드 {idx+1} / 폴드 {folds.n_splits}', '#'*40)
    
    X_train, y_train = X[train_idx], y[train_idx]
    X_valid, y_valid = X[valid_idx], y[valid_idx]
    
    dtrain = Pool(X_train, y_train, cat_features=None)
    dvalid = Pool(X_valid, y_valid, cat_features=None)
    
    cat_model = CatBoostClassifier(**max_params)
    cat_model.fit(dtrain, eval_set=dvalid, early_stopping_rounds=200, verbose=500)
    
    # 테스트 데이터 예측
    oof_test_preds += cat_model.predict_proba(X_test)[:, 1] / folds.n_splits
    # 검증 데이터 예측
    oof_val_preds[valid_idx] += cat_model.predict_proba(X_valid)[:, 1]
    
    # 검증 데이터 ROC-AUC 계산
    roc_auc = roc_auc_score(y_valid, oof_val_preds[valid_idx])
    print(f'폴드 {idx+1} ROC-AUC : {roc_auc}\n')

######################################## 폴드 1 / 폴드 10 ########################################
0:	test: 0.7122221	best: 0.7122221 (0)	total: 42.7ms	remaining: 1m 46s
500:	test: 0.7398299	best: 0.7398693 (496)	total: 18.2s	remaining: 1m 12s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.7400232145
bestIteration = 668

Shrink model to first 669 iterations.
폴드 1 ROC-AUC : 0.7400232144858506

######################################## 폴드 2 / 폴드 10 ########################################
0:	test: 0.7085172	best: 0.7085172 (0)	total: 36.1ms	remaining: 1m 30s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.7389539804
bestIteration = 287

Shrink model to first 288 iterations.
폴드 2 ROC-AUC : 0.7389539804133124

######################################## 폴드 3 / 폴드 10 ########################################
0:	test: 0.7049888	best: 0.7049888 (0)	total: 36ms	remaining: 1m 30s
500:	test: 0.7340744	best: 0.7342357 (444)	total: 17.9s	remaining: 1m 11s
Stopped 

## 5. 최종 결과

In [31]:
print('최종 OOF 검증 데이터 ROC-AUC:', roc_auc_score(y, oof_val_preds))

최종 OOF 검증 데이터 ROC-AUC: 0.7400634862086121


In [32]:
submission['probability'] = oof_test_preds
submission.to_csv('catboost_submission.csv', index=False)

In [33]:
submission = pd.read_csv('/kaggle/working/catboost_submission.csv')
submission.head()

,probability
0,0.001627
1,0.013028
2,0.158113
3,0.107663
4,0.530253
